In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import MNIST

In [81]:
dataset = MNIST(root="dataDig/", download = True)

In [82]:

import matplotlib.pyplot as plt
%matplotlib inline
import torchvision.transforms as transforms
from PIL import Image,ImageOps
import numpy

In [83]:
class Invert(object):
    
    def __call__(self, img):
        """
        :param img: PIL): Image 

        :return: Normalized image
        """
        img = numpy.asarray(img)
        img = 255 - img
        return Image.fromarray(img)

    def __repr__(self):
        return self.__class__.__name__+'()'

In [84]:
dataset = MNIST(root="dataDig/", train = True, transform=transforms.Compose([Invert() ,transforms.ToTensor()]))

In [85]:
#from torch.utils.data import random_split
train_loader = DataLoader(dataset,batchsize,shuffle=False)
#train_dataset, val_dataset = random_split(dataset, [60000,0])
batchsize = 100
#val_loader = DataLoader(val_dataset,batchsize,shuffle=False)
input_size = 28*28
num_classes = 10

In [86]:
class Net_DIGITS(nn.Module):   
    def __init__(self):
        super(Net_DIGITS, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear = nn.Sequential(
            nn.Linear(4*6*6,num_classes),
            nn.Softmax(),
        )


    # Defining the forward pass    
    def forward(self, x):
        #print(x.shape)
        x = x.reshape(-1,1,28,28)
        #print(x.shape)
        x = self.cnn_layers(x)
        #print(x.shape)
        x = x.reshape(-1,4*6*6)
        x = self.linear(x)
        return x

    def training_step(self, batch):
        images,labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels)
        return loss
    
    """
    def validation_step(self, batch):
        images,labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    """

model = Net_DIGITS()

In [87]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

loss_fn = F.cross_entropy
"""
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)
"""

'\ndef evaluate(model, val_loader):\n    outputs = [model.validation_step(batch) for batch in val_loader]\n    return model.validation_epoch_end(outputs)\n'

In [88]:
def fit(epochs, lr, model, train_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = []

    for epoch in range(epochs):

        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        #result = evaluate(model, val_loader)
        #model.epoch_end(epoch, result)
        #history.append(result)

    return history

In [89]:
fit(100,0.01,model,train_loader)


/Users/rushi/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[]

In [90]:
torch.save(model.state_dict(),'Test_dig.pt')